In [32]:
import numpy as np
import pandas as pd
import re

import pyspark
import databricks.koalas as ks
from pyspark.ml.evaluation import RegressionEvaluator
import pyspark.sql.functions as F
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import StringType, IntegerType
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from batcave.scrape_and_clean import new_id_dictionary, get_missing_titles
from batcave.recommend import get_recommendations, get_user_reviews_testing

In [33]:
spark = (pyspark.sql.SparkSession.builder
    .master("local")
    .getOrCreate())

### Loading comic and movie reviews

In [34]:
comic_reviews = ks.read_json('data/comic_reviews_wtitle.json')
movie_reviews = ks.read_json('data/movies_and_videos_wtitle.json')

### Creating new ids
Since the Amazon user and item ids contain both letters and numbers, I needed to give them new values for a couple reasons:
* The Spark ALS model will only take labels that are numeric and will cause an error otherwise.
* I can create ids for the comic books and movies/tv that are easily identifiable and easy to filter on.

In [6]:
new_user_ids = new_id_dictionary(comic_reviews, 'reviewerID', '00')
new_comic_asins = new_id_dictionary(comic_reviews, 'asin', '22')
new_mtv_asins = new_id_dictionary(movie_reviews, 'asin', '44')

In [7]:
all_items_asins = new_comic_asins
all_items_asins.update(new_mtv_asins)

In [ ]:
comic_reviews['item_id'] = comic_reviews.asin.map(all_items_asins)

In [10]:
udfUserId = F.udf(lambda x: new_user_ids[x], StringType())
udfItemId = F.udf(lambda x: all_items_asins[x], StringType())

In [11]:
comic_reviews_updated = comic_reviews.withColumn("item_id", udfItemId("asin"))
comic_reviews_updated = comic_reviews_updated.withColumn("user_id", udfUserId("reviewerID"))

In [12]:
comic_reviews_updated.show(1)

+----------+--------------------+-------+-------------+--------------------+-------+-------+
|      asin|               imUrl|overall|   reviewerID|               title|item_id|user_id|
+----------+--------------------+-------+-------------+--------------------+-------+-------+
|0345507460|http://ecx.images...|    5.0|ACO26JQ366659|The Dresden Files...| 236422| 771300|
+----------+--------------------+-------+-------------+--------------------+-------+-------+
only showing top 1 row



In [13]:
movie_reviews_updated = movie_reviews.withColumn("item_id", udfItemId("asin"))
movie_reviews_updated = movie_reviews_updated.withColumn("user_id", udfUserId("reviewerID"))

In [14]:
movie_reviews_updated.show(1)

+----------+--------------------+-------+-------------+--------------------+-------+-------+
|      asin|               imUrl|overall|   reviewerID|               title|item_id|user_id|
+----------+--------------------+-------+-------------+--------------------+-------+-------+
|0767807693|http://ecx.images...|    3.0|ADENUJJYKNHPO|Requiem for a Hea...|1464744| 794500|
+----------+--------------------+-------+-------------+--------------------+-------+-------+
only showing top 1 row



### Narrowing my dataset
To help improve the accuracy of my model, I wanted to look through removing some of the items and users that may not offer much insight because of their sparsity. Here I explored some of these features to help me decide on what to do.

#### Removing users who have rated less than average in either comics or movies

In [15]:
query = """
SELECT 
    item_id 
,   COUNT(*) as count 
FROM 
    table 
GROUP BY item_id
ORDER BY count desc"""

In [16]:
comic_reviews_updated.createOrReplaceTempView('table')
comic_low_reviewers = spark.sql(query).toPandas()

In [17]:
low_reviews = comic_low_reviewers[comic_low_reviewers['count'] \
                                  <= comic_low_reviewers['count'].mean()]
print(f"Amount of items with less than average amount of comic reviews: {low_reviews.shape[0]}")

Amount of items with less than average amount of comic reviews: 4129


In [18]:
movie_reviews_updated.createOrReplaceTempView('table')
mtv_low_reviewers = spark.sql(query).toPandas()

In [19]:
low_mtv_reviews = mtv_low_reviewers[mtv_low_reviewers['count'] \
                                  <= mtv_low_reviewers['count'].mean()]
print(f"Amount of items with less than average amount of movie reviews: {low_mtv_reviews.shape[0]}")

Amount of items with less than average amount of movie reviews: 28082


In [20]:
mtv_low_reviewers.head()

,item_id,count
0,5444,170
1,39044,163
2,2077544,141
3,2097544,140
4,2326944,133


In [21]:
low_item = list(set(low_reviews['item_id'].tolist() + low_mtv_reviews['item_id'].tolist()))
len(low_item)

32211

In [22]:
# Joining and adding review counts
all_review_counts_df = pd.concat([mtv_low_reviewers,comic_low_reviewers])
all_review_counts = spark.createDataFrame(all_review_counts_df)

# Joining comic & movie data
all_reviews = comic_reviews_updated.union(movie_reviews_updated)

# Adding the count column
all_reviews = all_reviews.join(all_review_counts, on='item_id', how='left')

In [23]:
all_reviews.show(5)

+-------+----------+--------------------+-------+--------------+--------------------+-------+-----+
|item_id|      asin|               imUrl|overall|    reviewerID|               title|user_id|count|
+-------+----------+--------------------+-------+--------------+--------------------+-------+-----+
|1003644|6301661192|http://ecx.images...|    1.0|A1D2C0WDCSHUWZ|      Meridian [VHS]|  81600|    1|
| 101122|0785115277|http://ecx.images...|    2.0|A169I83JL8QJGN|Excalibur Vol. 1:...| 824100|    5|
| 101122|0785115277|http://ecx.images...|    1.0|A1AM5YLSD7H0CM|Excalibur Vol. 1:...| 205000|    5|
| 101122|0785115277|http://ecx.images...|    4.0| AJKWF4W7QD4NS|Excalibur Vol. 1:...| 702000|    5|
| 101122|0785115277|http://ecx.images...|    1.0| AJ8L5D48CSEB4|Excalibur Vol. 1:...| 683800|    5|
+-------+----------+--------------------+-------+--------------+--------------------+-------+-----+
only showing top 5 rows



In [24]:
all_reviews_ready = all_reviews.filter(F.col('item_id').isin(low_item) == False)

In [25]:
# Seeing the unique count of users and items
all_reviews_ready.agg(*(F.countDistinct(F.col(c)).alias(c) for c in all_reviews_ready.columns)).show()

+-------+----+-----+-------+----------+-----+-------+-----+
|item_id|asin|imUrl|overall|reviewerID|title|user_id|count|
+-------+----+-----+-------+----------+-----+-------+-----+
|   7872|7872| 7362|      5|      8565| 5293|   8565|  105|
+-------+----+-----+-------+----------+-----+-------+-----+



In [26]:
# Exporting to temporarily preserve
all_reviews_ready.repartition(1).write.json("data/all_reviews_new")

### Case of the missing titles
Even though I had limited my set to movies/tv that have metadata, I found that some of the metadata is missing the title for movies, which is a pretty important piece for my further development!  Since I do have the ASINs, I wrote a quick function to find those missing titles by querying Amazon and then returning the first only results title. My process was as follows:
* Get the ASINs for data missing titles
* Run function on those ASINs to return a title
* Do some clean up of titles
* Add titles back to original data and drop old listing
* Export all_reviews again

In [27]:
all_reviews = spark.read.json('data/all_reviews_new.json')

In [28]:
# Getting the ASINS from products missing titles
missing_titles = all_reviews.select(['asin','title']).toPandas()

missing_asins = list(set(missing_titles.loc[missing_titles['title'].isna(), 'asin'].tolist()))

In [29]:
print(f" Records missing name: {len(missing_asins)}")

 Records missing name: 2556


In [30]:
# Using a function to scrape Amazon for the title - Takes a few hours to complete
missing_title_info = get_missing_titles(missing_asins)

In [31]:
# Drop these into a dataframe to inspect
missing_df = pd.DataFrame(missing_title_info)

In [32]:
missing_df.head()

,asin,title
0,B004MSP04C,None found
1,B002IVNLKA,World's Greatest Dad
2,B002ZG99WW,Yogi Bear (2010)
3,B000OCY7JE,After the Wedding
4,B003ND0AXS,Disney Phineas & Ferb: A Very Perry Christmas


In [33]:
missing_df.to_csv('data/missing_titles.csv')

In [34]:
missing_df = pd.read_csv('data/missing_titles.csv')

In [35]:
# Still had some that returned no name, so ignoring for now and will drop from all reviews
all_the_good = missing_df[missing_df['title'] != 'None found']

In [36]:
#Setting ASIN as index to do replace with original set
all_the_good.set_index('asin', inplace=True)
missing_titles.set_index('asin', inplace=True)
all_the_good.head()

,Unnamed: 0,title
asin,,
B002IVNLKA,1,World's Greatest Dad
B002ZG99WW,2,Yogi Bear (2010)
B000OCY7JE,3,After the Wedding
B003ND0AXS,4,Disney Phineas & Ferb: A Very Perry Christmas
B003L77G7Y,5,"American Dad!, Vol. 6"


In [37]:
# Combining to replace missing titles
missing_titles = missing_titles.combine_first(all_the_good)

# Resetting the index and changing title column name 
missing_titles.reset_index(inplace=True)

In [38]:
# Getting a list of all without a title still
check_missing_asins = list(set(missing_df.loc[missing_titles['title'].isna(), 'asin'].tolist()))

# Dropping rows with reviews that have no title from original dataframe
all_reviews_less_missing_titles = all_reviews.filter(F.col('asin').isin(check_missing_asins)==False)

# Dropping rows from my temporary dataframe with correct names
fix_titles = missing_titles.drop(missing_titles[missing_titles['title'].isna()].index)

In [39]:
# Some text cleanup on titles. There may be more later, but these are the initial examples I found:
potential_regs = """\[VHS\]|\[DVD\]|Collector\'s Edition|\: Season \d+
                    |\: The Complete Series|\: Complete Series|\(.*\)"""

fix_titles['title'] = fix_titles['title']\
                         .apply(lambda x: re.sub(potential_regs, '', x))

In [40]:
fix_titles.drop('Unnamed: 0', axis=1, inplace=True)
fix_titles.head()

,asin,title
0,0005119367,Joseph
1,0005119367,Joseph
2,0005119367,Joseph
3,0005119367,Joseph
4,0307142469,Frosty the Snowman


In [41]:
# Create Spark dataframe from dataframe with corrected titles
fixed_titles_spark = spark.createDataFrame(fix_titles)

In [42]:
# Combining & exporting
all_reviews_no_title = all_reviews_less_missing_titles.select(['asin',
                                                               'count',
                                                               'imUrl',
                                                               'item_id',
                                                               'overall',
                                                               'reviewerID',
                                                               'user_id'])

all_reviews_with_fixed_titles = all_reviews_no_title.join(fixed_titles_spark,
                                                                     on='asin',
                                                                     how='left').dropDuplicates()

In [43]:
all_reviews_with_fixed_titles.show(10)

+----------+-----+--------------------+-------+-------+--------------+-------+--------------------+
|      asin|count|               imUrl|item_id|overall|    reviewerID|user_id|               title|
+----------+-----+--------------------+-------+-------+--------------+-------+--------------------+
|0345507460|   57|http://ecx.images...| 236422|    5.0| ACO26JQ366659| 771300|The Dresden Files...|
|0345507460|   57|http://ecx.images...| 236422|    5.0|A34C35QFA4DC5J| 808700|The Dresden Files...|
|0345507460|   57|http://ecx.images...| 236422|    5.0|A3TII4RKU0ZVT4| 446700|The Dresden Files...|
|0345507460|   57|http://ecx.images...| 236422|    5.0|A1LR4Z5Z0MPYIF| 818200|The Dresden Files...|
|0345507460|   57|http://ecx.images...| 236422|    5.0|A16L43DIFSHGMQ| 758300|The Dresden Files...|
|0345507460|   57|http://ecx.images...| 236422|    5.0| ANRS196NKFVUU|  53100|The Dresden Files...|
|0345507460|   57|http://ecx.images...| 236422|    1.0|A30BEBBQ3UYI7O| 611900|The Dresden Files...|


In [44]:
all_reviews_with_fixed_titles.repartition(1).write.json('data/all_reviews_fixed_titles')

### Modeling and testing

In [35]:
all_reviews  = spark.read.json('data/all_reviews_fixed_titles.json')

In [36]:
als_ready = all_reviews.select([F.col("user_id").cast(IntegerType()),
                                  F.col("item_id").cast(IntegerType()),
                                  F.col("overall").cast(IntegerType()),
                                  F.col("title"), 
                                  F.col("imUrl")])

In [ ]:
# Build the recommendation model using ALS
als = ALS(rank=50, regParam=.1, maxIter=20,
          userCol='user_id', itemCol='item_id', 
          ratingCol='overall', nonnegative=True)

als_model = als.fit(als_ready)

In [49]:
test_pred_df = als_model.transform(test).toPandas()

# Filling in NaN values with average score for metric review
test_pred_df['prediction'].fillna(4, inplace=True)
test_pred = spark.createDataFrame(test_pred_df)

# Get RMSE & MAE for model
evaluator = RegressionEvaluator(metricName="rmse", labelCol="overall",
                                predictionCol="prediction")

evaluator_2 = RegressionEvaluator(metricName="mae", labelCol="overall",
                                predictionCol="prediction")

rmse_test = evaluator.evaluate(test_pred)
mae_test = evaluator_2.evaluate(test_pred)
print(f"Test RMSE: {rmse_test}")
print(f"Test MAE: {mae_test}")

Test RMSE: 1.1743178289845608
Test MAE: 0.9355244490473343


### Parameter tuning for optimization
I kept running into errors with my parameter grid to cross validate below, so ran several tests as well varying parameters and landed on my best model using the configuration found in the above model test.  Below is an example of the parameters this was based on.

In [ ]:
als = ALS(userCol='user_id', itemCol='item_id', ratingCol='overall', nonnegative=True)

reg_test = RegressionEvaluator(predictionCol='prediction', labelCol='overall')

# Parameter grid              
params = ParamGridBuilder().addGrid(als.regParam, [0.01,0.001,0.1])\
                           .addGrid(als.rank, [4,10,50])\
                           .addGrid(als.maxIter, [5,10,15,20]).build()
             
## Calling and checking evaluator
cv = CrossValidator(estimator=als, estimatorParamMaps=params,evaluator=reg_test,parallelism=4)
best_model = cv.fit(train)

### Returning recommendations
To get recommendations for new users, I first need to get the item features from my ALS model, save them with titles, and then can use that file to create and build recommendations.
Below is an example of the two functions created to perform the recommendation operation, just on the Notebook level:
* ```get_user_reviews_testing```: gives an individual a selection of movies to choose from, once they have rated at least 10, it creates and returns a dataframe with their scores.
* ```get_recommedations```: returns the top 10 comic books recommended to the user based on their input.

These functions will be used as the basis for creating a functional web application for users to explore.

In [37]:
# Build the recommendation model using ALS
als = ALS(rank=50, regParam=.1, maxIter=20,
          userCol='user_id', itemCol='item_id', 
          ratingCol='overall', nonnegative=True)

als_model = als.fit(als_ready)

In [38]:
# Get item factors from model
item_factors = als_model.itemFactors.toPandas()
item_factors.rename(columns={'id': 'item_id'}, inplace=True)
item_factors['item_id'] = item_factors['item_id'].astype(str)

In [39]:
# Get titles from original dataset
item_titles = all_reviews.select(['item_id', 'title', 'count', 'asin']).distinct().toPandas()
item_titles['item_id'] = item_titles['item_id'].astype(str)

In [40]:
# Merge into one dataframe & making the ids string for exploration later
item_details = item_factors.merge(item_titles, on='item_id', how='left')

In [ ]:
item_details

In [41]:
# Exporting to preserve item factor details
export_df = spark.createDataFrame(item_details)
export_df.repartition(1).write.csv('data/als_item_factor_details')

AnalysisException: 'CSV data source does not support array<double> data type.;'

In [15]:
item_details.shape

(7872, 5)

In [17]:
user_test = get_user_reviews_testing(item_details)

In [18]:
user_test.head()

,item_id,rating
0,715044,4
1,2934544,3
2,858944,4
3,2306644,2
4,1453244,1


In [19]:
recs = get_recommendations(item_factors_df=item_details, new_user_df=user_test)

In [20]:
recs

,item_id,title,asin,new_user_predictions
462,183222,Batman Illustrated by Neal Adams: Volume 3,1401204074,4.855964
637,255522,Ultimate X-Men Vol. 1: The Tomorrow People,0785107886,4.512662
496,198322,Captain America: The Art of Captain America - ...,0785155090,4.232017
295,119222,Secret Wars,078511873X,4.187856
447,177222,Batman: Vampire,1401215653,4.152080


Looking good! This finishes my data prep and modeling phase.

### Understanding how to fold a new user into the model
Below is a walkthrough of getting recommendations for a user by using the existing item features and the ratings from the new user. 

In [21]:
# Getting item features from the model and setting the index to the item_id
item_factors = als_model.itemFactors.toPandas()
item_factors.index = item_factors['id']
item_factors['id'] = item_factors['id'].astype(str)

In [22]:
# Looking for some movies to rate, all movies id ends in 44
item_factors.loc[item_factors['id'].str.endswith('44')].head()

,id,features
id,,
144,144,"[0.27627408504486084, 0.3660491704940796, 0.22..."
1344,1344,"[0.21467094123363495, 0.25732994079589844, 0.0..."
1544,1544,"[0.3672632575035095, 0.28194597363471985, 0.35..."
2044,2044,"[0.28777071833610535, 0.301589697599411, 0.131..."
2244,2244,"[0.08827913552522659, 0.32467201352119446, 0.1..."


In [23]:
# Creating a random user with a set of ratings
user = [{'id': 144, 'rating': 3}, {'id': 1344, 'rating': 3}, 
        {'id': 1544, 'rating': 3}, {'id': 2244, 'rating': 3}]
user_df = pd.DataFrame(user)
user_df

,id,rating
0,144,3
1,1344,3
2,1544,3
3,2244,3


In [24]:
# Create lists of each column to calculate user matrix
item_ids = user_df.id.tolist()
user_rating = user_df.rating.tolist()

In [25]:
# User ratings 
all_ratings_array = np.array((user_rating,)).T

# Get item features for these specific movies
all_items_array = np.zeros(shape=(len(item_ids), 50))

for index, item in enumerate(item_ids):
    all_items_array[index, :] = np.array(item_factors.loc[item, 'features'])

In [26]:
# Checking the shape of each to make sure things are looking right
all_ratings_array.shape, all_items_array.shape

((4, 1), (4, 50))

In [27]:
# Least squares solution to get user features
new_user_matrix = np.linalg.lstsq(all_items_array, all_ratings_array, rcond=None)

# New users matrix!
new_user_matrix = new_user_matrix[0].reshape((50,))
new_user_matrix.shape

(50,)

In [28]:
# Checking based on known score
known_item = np.array(item_factors.loc[144,'features'])

score = np.dot(new_user_matrix, known_item)
score

2.999999999999999

Nice! Now I am going to user that factor dataframe to create predictions for this user

In [29]:
item_factors['new_user_predictions'] = item_factors['features'].apply(lambda x: np.dot(x, new_user_matrix))

In [30]:
top_five_comics = item_factors.loc[item_factors['id']\
                              .str.endswith('22'), 'new_user_predictions']\
                              .sort_values(ascending=False)[:5]

In [31]:
top_five_comics

id
311422    3.620749
112922    3.546200
89922     3.486923
315022    3.477472
70322     3.475644
Name: new_user_predictions, dtype: float64

Perfect! I can now functionize this principle and apply to getting speedy recommendations for my new users.